In [1]:
import pandas as pd
import os
import glob
import re
import torch
from torch.utils.data import random_split

os.chdir('../')

from src.features.preprocessing import SoundDS
from src.models.model import AudioClassifier
from src.models.train_model import training
from src.models.predict_model import inference



In [2]:

files = glob.glob('data/raw/*/*.wav')
classes = [re.search('raw/(.+?)/', file).group(1) for file in files]

df = pd.DataFrame({'relative_path': files, 'class': classes})
codes, uniques = pd.factorize(df['class'])
df['class_id'] = codes
df.head()

,relative_path,class,class_id
0,data/raw/Poecile_montanus/PoeMon00009.wav,Poecile_montanus,0
1,data/raw/Poecile_montanus/PoeMon00008.wav,Poecile_montanus,0
2,data/raw/Poecile_montanus/PoeMon00006.wav,Poecile_montanus,0
3,data/raw/Poecile_montanus/PoeMon00012.wav,Poecile_montanus,0
4,data/raw/Poecile_montanus/PoeMon00013.wav,Poecile_montanus,0


In [7]:
n_classes = df['class_id'].nunique()
n_classes

90

In [3]:
myds = SoundDS(df, data_path='')

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

print(train_ds)

In [4]:
# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

print(train_dl)

In [5]:
# Create the model and put it on the GPU if available
myModel = AudioClassifier(n_classes=n_classes)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device


device(type='cpu')

In [6]:
num_epochs=10   # Just for demo, adjust this higher.
training(myModel, device, train_dl, num_epochs)

Epoch: 0, Loss: 4.45, Accuracy: 0.04
Epoch: 1, Loss: 4.17, Accuracy: 0.09


KeyboardInterrupt: 

In [ ]:
# Run inference on trained model with the validation set
inference(myModel, device, val_dl)